<a href="https://colab.research.google.com/github/goin2crazy/arabic-text-classification/blob/main/lenta_ru_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Model

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
from torch.utils.data import DataLoader

from transformers import (AutoTokenizer,
                          AutoModelForTokenClassification,
                          AutoModelForSequenceClassification,
                          pipeline
                         )

from tqdm import tqdm

tqdm.pandas()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
class Config:
    datapath = '/kaggle/input/lenta-ru-private-dataset-for-tatar-hackathon/lenta_ru_news_2019_2023.csv'

    ner_preset = "Davlan/distilbert-base-multilingual-cased-ner-hrl"
    sentiment_preset = 'doublecringe123/lenta-ru-sentiments'

cfg = Config()

In [ ]:
class NERModel:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained(cfg.ner_preset)
        self.model = AutoModelForTokenClassification.from_pretrained(cfg.ner_preset).to(device)
        self.nlp = pipeline("ner", model=self.model, tokenizer=self.tokenizer, aggregation_strategy="max", device = device)

    def __call__(self, batch):
        return self.nlp(batch)

class SentimentModel:
    def __init__(self):
        model_checkpoint = cfg.sentiment_preset
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to(device)


    def get_sentiment(self, text, focus = ''):
        """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
        with torch.no_grad():
            inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
            outputs = self.model(**inputs).logits

            probas = torch.sigmoid(outputs).cpu().numpy()

        for proba in probas:
              yield proba.dot([-1, 0, 1])

    def __call__(self, *args, **kwargs):
        return self.get_sentiment( *args, **kwargs)

In [ ]:
ner_model = NERModel()
sentiment_model = SentimentModel()

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.1M [00:00<?, ?B/s]

In [ ]:
class Inference:
    def __init__(self):
        self.ner_model = NERModel()
        self.sentiment_model = SentimentModel()

    def predict_sentiment(self, word, text):
        """

        Args:
          word: focus word
          text: text to analize

        Returns:

        """
        p = list(self.sentiment_model(f"[focus: {word}] {text}"))

        return sum(p)/len(p)

    def add_sentiments(self, ner_text, text):
        text_sentences = text.split('.')

        ner_sentiments = {i['word']: self.predict_sentiment(i['word'], text) for i in ner_text}

        ner_sentiments = [{**nt, "sentiment": v} for nt, (k, v) in zip(ner_text, ner_sentiments.items())]
        return  ner_sentiments

    def process(self, batch):
        """
        Arguments:
            batch: list is strs
        """

        ner_batch = self.ner_model(batch)

        return [self.add_sentiments(ner_text, text) for ner_text, text in zip(ner_batch, batch)]

    def __call__(self, batch):
        if type(batch) == str:
            batch = [batch]

        return self.process(batch)

# Repo


In [ ]:
! git clone https://github.com/goin2crazy/lenta_ru.git

Cloning into 'lenta_ru'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 26 (delta 9), reused 23 (delta 8), pack-reused 0
Receiving objects: 100% (26/26), 10.87 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
! pip install -q /content/lenta_ru

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framewor

In [ ]:
# prompt: import Inference from lenta_ru

from lenta_ru import Inference

In [ ]:
inf = Inference()

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.1M [00:00<?, ?B/s]

In [ ]:
# sentiment > 0 - positive
# sentiment < 0 - negative

inf('I love working at Apple!')

[[{'entity_group': 'ORG',
   'score': 0.99978536,
   'word': 'Apple',
   'start': 18,
   'end': 23,
   'sentiment': 0.40757113695144653}]]